In [1]:
import numpy as np 
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from torchvision import transforms

In [2]:
keypoints = np.load("../data/keypoints/arrays.npz")
data_train = pd.read_csv("../data/revalidated/data_train_f_001.csv")
data_train.head()

,path,label,conf,revalidated_manually
0,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
1,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1
2,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.444923,1
3,.\data\train\subject-1\fall\1_backward_falls\f...,1,0.616470,1
4,.\data\train\subject-1\fall\1_backward_falls\f...,0,0.000000,1


In [3]:
features = []
for i in range(4294):
    if len(keypoints[f'arr_{i}']) > 0:
        features.append(keypoints[f'arr_{i}'])
    else:
        features.append(np.zeros(99))
features = np.array(features)
labels = data_train['label'].values

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
rf = RandomForestClassifier(n_estimators=50, criterion='log_loss', random_state=42)

In [14]:
rf.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss', n_estimators=50, random_state=42)

In [15]:
predictions = rf.predict(X_val)

In [16]:
from sklearn.metrics import classification_report

In [17]:
print(classification_report(y_val, predictions))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       546
           1       1.00      0.90      0.95       313

    accuracy                           0.96       859
   macro avg       0.97      0.95      0.96       859
weighted avg       0.96      0.96      0.96       859



In [12]:
sys.path.append(os.path.join(os.getcwd(), ".."))

In [18]:
from transformer import PoseKeypointsTransformer

In [19]:
transformer = PoseKeypointsTransformer(with_visibility=False)

In [20]:
test_data = pd.read_csv("../data/sample_submission.csv")

In [21]:
test_features = transformer(image_paths=[os.path.join("data", "test", path) for path in test_data.iloc[:, 0].values])

In [32]:
features = []
for i in range(len(test_features)):
    if len(test_features[i]) > 0:
        features.append(test_features[i])
    else:
        features.append(np.zeros(99))
features = np.array(features)
labels = test_data['label'].values

In [37]:
test_data['label'] = rf.predict(features)

In [39]:
test_data[test_data['label'] == 1]

,id,label
5,ffd7ae3605.jpg,1
7,37f3f2e49f.jpg,1
8,22a87835c1.jpg,1
12,c4a2e75cc7.jpg,1
13,5e6693247f.jpg,1
...,...,...
2131,9fd17b3752.jpg,1
2133,617ad0f5d2.jpg,1
2134,ea978b893c.jpg,1
2137,af4094a12e.jpg,1


In [40]:
test_data.to_csv("../data/submissions/008_rf.csv", index=False)